<a href="https://colab.research.google.com/github/Manuela-AYO/AirBnb/blob/main/Airbnb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Downloading the Dataset

In [ ]:
%%capture
! pip install numpy pandas streamlit gdown pyarrow

In [ ]:
import os
import shutil

import gdown
import numpy as np
import pandas as pd

# Download files from Google Drive
# Based on data from: http://insideairbnb.com/get-the-data/
file_id_1 = "1m185vTdh-u7_A2ZElBvUD4SCO6oETll2"
file_id_2 = "1w41V1oWHJrBdaNJJQ4oxVBuml5CO7MQX"
downloaded_file_1 = "listings_project.pkl"
downloaded_file_2 = "calendar_project.parquet"
# Download the files from Google Drive
gdown.download(id=file_id_1, output=downloaded_file_1)
gdown.download(id=file_id_2, output=downloaded_file_2)

Downloading...
From: https://drive.google.com/uc?id=1m185vTdh-u7_A2ZElBvUD4SCO6oETll2
To: /content/listings_project.pkl
100%|██████████| 1.42M/1.42M [00:00<00:00, 50.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1w41V1oWHJrBdaNJJQ4oxVBuml5CO7MQX
To: /content/calendar_project.parquet
100%|██████████| 1.23M/1.23M [00:00<00:00, 69.7MB/s]


'calendar_project.parquet'

In [ ]:
# Show all columns (instead of cascading columns in the middle)
pd.set_option("display.max_columns", None)
# Don't show numbers in scientific notation
pd.set_option("display.float_format", "{:.2f}".format)

## Preprocessing the Dataset

In [ ]:
# let's read the pickle and parquet
df_list = pd.read_pickle(downloaded_file_1)
df_cal = pd.read_parquet("calendar_project.parquet")

In [ ]:
df_list.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6165 entries, 0 to 6172
Data columns (total 34 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    6165 non-null   int64  
 1   host_acceptance_rate                  5365 non-null   float64
 2   host_is_superhost                     6165 non-null   object 
 3   host_listings_count                   6165 non-null   int64  
 4   host_total_listings_count             6165 non-null   int64  
 5   neighbourhood_cleansed                6165 non-null   object 
 6   latitude                              6165 non-null   float64
 7   longitude                             6165 non-null   float64
 8   room_type                             6165 non-null   object 
 9   accommodates                          6165 non-null   int64  
 10  bedrooms                              5859 non-null   float64
 11  beds             

In [ ]:
df_list.head()

,id,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,neighbourhood_cleansed,latitude,longitude,room_type,accommodates,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,review_scores_rating,instant_bookable,reviews_per_month,price_in_euros,price_per_person,minimum_price,discount_per_5_days_booked,discount_per_10_days_booked,discount_per_30_and_more_days_booked,service_cost
0,23726706,0.95,f,1,1,IJburg - Zeeburgereiland,52.35,4.98,Private room,2,1.00,1.00,6,$88.00,2,14,t,0,6,20,66,78,11,3,4.99,t,1.53,None,$44,$176,5%,11%,16%,$4.99
1,35815036,1.00,t,1,1,Noord-Oost,52.42,4.96,Entire home/apt,2,NaN,1.00,5,$105.00,3,100,t,4,6,12,243,95,36,6,4.96,f,2.65,None,$52.5,$315,5%,12%,16%,$4.99
2,31553121,1.00,f,1,1,Noord-West,52.43,4.92,Entire home/apt,4,1.00,3.00,3,$152.00,2,60,t,0,3,3,3,82,26,1,4.74,f,2.02,None,$38,$304,7%,11%,22%,$4.99
3,34745823,0.94,f,3,3,Gaasperdam - Driemond,52.30,5.01,Entire home/apt,2,1.00,2.00,8,$87.00,2,1125,t,5,20,26,290,39,4,0,4.87,f,1.08,None,$43.5,$174,6%,10%,15%,$4.99
4,44586947,0.88,t,0,0,Gaasperdam - Driemond,52.31,5.03,Private room,4,2.00,3.00,4,$160.00,2,31,t,9,32,62,152,15,12,3,5.00,f,0.68,None,$40,$320,9%,22%,20%,$4.99


### Investigation through columns

In [ ]:
df_list.discount_per_5_days_booked.head(5)

0    5%
1    5%
2    7%
3    6%
4    9%
Name: discount_per_5_days_booked, dtype: object

In [ ]:
df_list.discount_per_10_days_booked.head(5)

0    11%
1    12%
2    11%
3    10%
4    22%
Name: discount_per_10_days_booked, dtype: object

In [ ]:
df_list.discount_per_30_and_more_days_booked.head(5)

0    16%
1    16%
2    22%
3    15%
4    20%
Name: discount_per_30_and_more_days_booked, dtype: object

In [ ]:
# data type transformation + formating
df_list["discount_per_5_days_booked"] = df_list["discount_per_5_days_booked"].str.replace("%", "", regex=True).astype("float")/100
df_list["discount_per_10_days_booked"] = df_list["discount_per_10_days_booked"].str.replace("%", "", regex=True).astype("float")/100
df_list["discount_per_30_and_more_days_booked"] = df_list["discount_per_30_and_more_days_booked"].str.replace("%", "", regex=True).astype("float")/100


Awesome! Let's inspect our results.

In [ ]:
df_list.discount_per_5_days_booked.head(5)

0   0.05
1   0.05
2   0.07
3   0.06
4   0.09
Name: discount_per_5_days_booked, dtype: float64

This data looks great for performing calculations!

Next, the columns `host_is_superhost`, `instant_bookable`, and `has_availability` are all boolean columns in the sense that their data represents true and false values, but currently are recognized as objects.

In [ ]:
df_list[["host_is_superhost", "instant_bookable", "has_availability"]].head(5)

,host_is_superhost,instant_bookable,has_availability
0,f,t,t
1,t,f,t
2,f,f,t
3,f,f,t
4,t,f,t


This is because the letters in these columns (**t** and **f**) are written as **strings** and not as **boolean** data types. This means we need to replace our string values with the boolean equivalent dtype.

Let's now check to confirm we executed these changes correctly.

In [ ]:
df_list[["host_is_superhost", "instant_bookable", "has_availability"]].head(5)

,host_is_superhost,instant_bookable,has_availability
0,False,True,True
1,True,False,True
2,False,False,True
3,False,False,True
4,True,False,True


A closer look at the prices in the four columns `price`, `price_per_person`, `minimum_price`, and `service_cost` reveals that they all follow the same pattern:

In [ ]:
df_list[["price", "price_per_person", "minimum_price", 'service_cost']].head(5)

,price,price_per_person,minimum_price,service_cost
0,$88.00,$44,$176,$4.99
1,$105.00,$52.5,$315,$4.99
2,$152.00,$38,$304,$4.99
3,$87.00,$43.5,$174,$4.99
4,$160.00,$40,$320,$4.99


In [ ]:
# reformating + changing data types
df_list["price"] = df_list["price"].str.replace("$", "", regex=True).str.replace(",", "", regex=True).astype("float")
df_list["price_per_person"] = df_list["price_per_person"].str.replace("$", "", regex=True).str.replace(",", "", regex=True).astype("float")
df_list["minimum_price"] = df_list["minimum_price"].str.replace("$", "", regex=True).str.replace(",", "", regex=True).astype("float")
df_list["service_cost"] = df_list["service_cost"].str.replace("$", "", regex=True).str.replace(",", "", regex=True).astype("float")

Let's inspect the different price columns again and see what it look like

In [ ]:
df_list[["price", "price_per_person", "minimum_price", 'service_cost']].head(5)

,price,price_per_person,minimum_price,service_cost
0,88.00,44.00,176.00,4.99
1,105.00,52.50,315.00,4.99
2,152.00,38.00,304.00,4.99
3,87.00,43.50,174.00,4.99
4,160.00,40.00,320.00,4.99


The following column names need to be changed:
- `price` into `price_in_dollar`
- `neighbourhood_cleansed` into `neighbourhood`  


In [ ]:
df_list = df_list.rename(columns = {"price" : "price_in_dollar", "neighbourhood_cleansed" : "neighbourhood"})

In [ ]:
df_list.columns

Index(['id', 'host_acceptance_rate', 'host_is_superhost',
       'host_listings_count', 'host_total_listings_count', 'neighbourhood',
       'latitude', 'longitude', 'room_type', 'accommodates', 'bedrooms',
       'beds', 'amenities', 'price_in_dollar', 'minimum_nights',
       'maximum_nights', 'has_availability', 'availability_30',
       'availability_60', 'availability_90', 'availability_365',
       'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d',
       'review_scores_rating', 'instant_bookable', 'reviews_per_month',
       'price_in_euros', 'price_per_person', 'minimum_price',
       'discount_per_5_days_booked', 'discount_per_10_days_booked',
       'discount_per_30_and_more_days_booked', 'service_cost'],
      dtype='object')

Taking a closer look at the `neighbourhood` and `room_type` columns reveals that these columns are assigned an **object** dtype. We want them to be a **category** dtype. Please set the correct data type below.

In [ ]:
df_list["neighbourhood"] = df_list["neighbourhood"].astype("category")
df_list["room_type"] = df_list["room_type"].astype("category")

We've made quite a few changes 👌, but we're not done yet! Next we need to delete any columns in the DataFrame that we won't use in our analysis. 

On this part, we will delete irrelevant columns

We need to delete some columns that are irrelevant to our current use case. Those irrelevant columns are:
* `host_listings_count`
* `host_total_listings_count`
* `availability_60`
* `availability_90`
* `availability_365`
* `number_of_reviews`
* `number_of_reviews_ltm`
* `reviews_per_month`

In [ ]:
df_list = df_list.drop(columns = ["host_listings_count", "host_total_listings_count", \
                       "availability_90", "availability_365", "number_of_reviews", \
                       "number_of_reviews_ltm", "reviews_per_month"])

In [ ]:
df_list.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6165 entries, 0 to 6172
Data columns (total 27 columns):
 #   Column                                Non-Null Count  Dtype   
---  ------                                --------------  -----   
 0   id                                    6165 non-null   int64   
 1   host_acceptance_rate                  5365 non-null   float64 
 2   host_is_superhost                     6165 non-null   bool    
 3   neighbourhood                         6165 non-null   category
 4   latitude                              6165 non-null   float64 
 5   longitude                             6165 non-null   float64 
 6   room_type                             6165 non-null   category
 7   accommodates                          6165 non-null   int64   
 8   bedrooms                              5859 non-null   float64 
 9   beds                                  6082 non-null   float64 
 10  amenities                             6165 non-null   int64   
 11  pric

Let's now have a look at which data types we still need to change and which columns have some null values.

In [ ]:
df_list.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6165 entries, 0 to 6172
Data columns (total 27 columns):
 #   Column                                Non-Null Count  Dtype   
---  ------                                --------------  -----   
 0   id                                    6165 non-null   int64   
 1   host_acceptance_rate                  5365 non-null   float64 
 2   host_is_superhost                     6165 non-null   bool    
 3   neighbourhood                         6165 non-null   category
 4   latitude                              6165 non-null   float64 
 5   longitude                             6165 non-null   float64 
 6   room_type                             6165 non-null   category
 7   accommodates                          6165 non-null   int64   
 8   bedrooms                              5859 non-null   float64 
 9   beds                                  6082 non-null   float64 
 10  amenities                             6165 non-null   int64   
 11  pric

We can see from the output above that the columns `host_acceptance_rate`, `review_scores_rating`, `bedrooms`, `beds`, and `price_in_euros` still require some processing, as they contain missing values, and/or have dtypes like object or float when they need an integer data type.

Let's inspect the `price_in_euros` column first, because this column seems to contain only null values, which inherently do not add any meaning to our dataset. 

In [ ]:
df_list["price_in_euros"].unique()

array([None], dtype=object)

In [ ]:
df_list = df_list.drop(columns = ["price_in_euros"])

In [ ]:
df_list.columns

Index(['id', 'host_acceptance_rate', 'host_is_superhost', 'neighbourhood',
       'latitude', 'longitude', 'room_type', 'accommodates', 'bedrooms',
       'beds', 'amenities', 'price_in_dollar', 'minimum_nights',
       'maximum_nights', 'has_availability', 'availability_30',
       'availability_60', 'number_of_reviews_l30d', 'review_scores_rating',
       'instant_bookable', 'price_per_person', 'minimum_price',
       'discount_per_5_days_booked', 'discount_per_10_days_booked',
       'discount_per_30_and_more_days_booked', 'service_cost'],
      dtype='object')

DataFrame [`info()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.info.html) revealed that some listings have no reviews and an unknown host acceptance rate. Most Airbnb users exclude such listings from their search results. To mimic this filtering approach, let us filter out any rows that do not have a `review_scores_rating` and without a `host_acceptance_rate`.

In [ ]:
df_list = df_list.dropna(subset=["review_scores_rating", "host_acceptance_rate"])

In [ ]:
df_list.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4886 entries, 0 to 6172
Data columns (total 26 columns):
 #   Column                                Non-Null Count  Dtype   
---  ------                                --------------  -----   
 0   id                                    4886 non-null   int64   
 1   host_acceptance_rate                  4886 non-null   float64 
 2   host_is_superhost                     4886 non-null   bool    
 3   neighbourhood                         4886 non-null   category
 4   latitude                              4886 non-null   float64 
 5   longitude                             4886 non-null   float64 
 6   room_type                             4886 non-null   category
 7   accommodates                          4886 non-null   int64   
 8   bedrooms                              4622 non-null   float64 
 9   beds                                  4817 non-null   float64 
 10  amenities                             4886 non-null   int64   
 11  pric

We also have some missing data on the room_type. There we will make some assumptions.

Let's try and make some simple assumptions based on the `room_type` assigned to the listing. First, inspect which room types are found in the dataset. 

In [ ]:
df_list["room_type"].unique()

['Private room', 'Entire home/apt', 'Hotel room', 'Shared room']
Categories (4, object): ['Entire home/apt', 'Hotel room', 'Private room', 'Shared room']

There are four room types. Let's make the assumption that the columns `bedrooms` and `beds` are potentially influenced by `room_type`.

Therefore, we can make the following rules:
- If you have a **"Private room"** or **"Shared room"** as `room_type`, then we believe the listing only has one bedroom. 
- If the listing has **"Hotel room"** or **"Entire home/apt"** as `room_type`, then we can divide the number of guests the listing accomodates by 2 and round up. 
- If any of these numbers are missing, then we can leave it empty.

In [ ]:
def fill_empty_bedrooms(accommodates: int, bedrooms: int, room_type: str) -> int:
    if (room_type == "Private room") or (room_type == "Shared room"):
        return 1
    elif (room_type == "Hotel room") or (room_type == "Entire home/apt"):
        return np.ceil(accommodates / 2)
    else:
        return bedrooms

In [ ]:
%%timeit -r 4 -n 100

temp_df = df_list.copy()  # Deep copy of the df, not a "view"
temp_df["rooms"] = df_list[["accommodates", "bedrooms", "room_type"]].apply(
    lambda x: fill_empty_bedrooms(x["accommodates"], x["bedrooms"], x["room_type"]),
    axis=1,
)

86 ms ± 5.81 ms per loop (mean ± std. dev. of 4 runs, 100 loops each)


In [ ]:
df_list["room_type"].unique()

['Private room', 'Entire home/apt', 'Hotel room', 'Shared room']
Categories (4, object): ['Entire home/apt', 'Hotel room', 'Private room', 'Shared room']

In [ ]:
df_list["bedrooms"] = df_list[["accommodates", "bedrooms", "room_type"]].apply(
    lambda x : fill_empty_bedrooms(x["accommodates"], x["bedrooms"], x["room_type"]), 
    axis = 1
)

| Related functions ([In *general* order of preference](https://stackoverflow.com/questions/24870953/does-pandas-iterrows-have-performance-issues/24871316#24871316))|
| ---- |
| [apply()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html): Apply a function along one or multiple columns |
| [pipe()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pipe.html): Chain multiple transformations/functions after each other |
| [applymap()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.applymap.html): Use strictly as a transformation of current value to a new value |
| [itertuples()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.itertuples.html): Iterate over DataFrame rows as named tuples |
| [iteritems()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iteritems.html): Iterate  over DataFrame columns |
| [iterrows()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iterrows.html): Iterate over DataFrame rows |

In [ ]:
df_list.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4886 entries, 0 to 6172
Data columns (total 26 columns):
 #   Column                                Non-Null Count  Dtype   
---  ------                                --------------  -----   
 0   id                                    4886 non-null   int64   
 1   host_acceptance_rate                  4886 non-null   float64 
 2   host_is_superhost                     4886 non-null   bool    
 3   neighbourhood                         4886 non-null   category
 4   latitude                              4886 non-null   float64 
 5   longitude                             4886 non-null   float64 
 6   room_type                             4886 non-null   category
 7   accommodates                          4886 non-null   int64   
 8   bedrooms                              4886 non-null   float64 
 9   beds                                  4817 non-null   float64 
 10  amenities                             4886 non-null   int64   
 11  pric

Just to measure the power of vectorization

In [ ]:
%%timeit -r 4 -n 100

temp_df = df_list.copy()


mask = temp_df["room_type"] == "Private room"
temp_df.loc[mask, "bedrooms"] = 1
mask = temp_df["room_type"] == "Shared room"
temp_df.loc[mask, "bedrooms"] = 1
mask = temp_df["room_type"] == "Entire home/apt" 
temp_df.loc[mask, "bedrooms"] = np.ceil(temp_df["accommodates"]/2)
mask = temp_df["room_type"] == "Hotel room"
temp_df.loc[mask, "bedrooms"] = np.ceil(temp_df["accommodates"]/2)

4.89 ms ± 2.18 ms per loop (mean ± std. dev. of 4 runs, 100 loops each)


In [ ]:
# let us remove all rows/entries that have an empty bedrooms, beds
df_list = df_list.dropna(subset=["beds"])

In [ ]:
df_list.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4817 entries, 0 to 6172
Data columns (total 26 columns):
 #   Column                                Non-Null Count  Dtype   
---  ------                                --------------  -----   
 0   id                                    4817 non-null   int64   
 1   host_acceptance_rate                  4817 non-null   float64 
 2   host_is_superhost                     4817 non-null   bool    
 3   neighbourhood                         4817 non-null   category
 4   latitude                              4817 non-null   float64 
 5   longitude                             4817 non-null   float64 
 6   room_type                             4817 non-null   category
 7   accommodates                          4817 non-null   int64   
 8   bedrooms                              4817 non-null   float64 
 9   beds                                  4817 non-null   float64 
 10  amenities                             4817 non-null   int64   
 11  pric

Now that we have removed all the empty values, finally we can assign the dtype `int` instead of `float` to these two columns.

In [ ]:
df_list[["beds", "bedrooms"]] = df_list[["beds", "bedrooms"]].astype("int")

In [ ]:
df_list.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4817 entries, 0 to 6172
Data columns (total 26 columns):
 #   Column                                Non-Null Count  Dtype   
---  ------                                --------------  -----   
 0   id                                    4817 non-null   int64   
 1   host_acceptance_rate                  4817 non-null   float64 
 2   host_is_superhost                     4817 non-null   bool    
 3   neighbourhood                         4817 non-null   category
 4   latitude                              4817 non-null   float64 
 5   longitude                             4817 non-null   float64 
 6   room_type                             4817 non-null   category
 7   accommodates                          4817 non-null   int64   
 8   bedrooms                              4817 non-null   int64   
 9   beds                                  4817 non-null   int64   
 10  amenities                             4817 non-null   int64   
 11  pric

---

#### Memory optimization

On this section, we'll assign the right number of bytes to our data

In [ ]:
df_list["service_cost"].max()

10.99

In [ ]:
df_list[["beds", "bedrooms", "accommodates", "amenities", "availability_30", "availability_60", "number_of_reviews_l30d"]] = \
df_list[["beds", "bedrooms", "accommodates", "amenities", "availability_30", "availability_60", "number_of_reviews_l30d"]].astype("int8")

df_list[["minimum_nights", "maximum_nights"]] = df_list[["minimum_nights", "maximum_nights"]].astype("int16")

df_list[["host_acceptance_rate", "review_scores_rating", "discount_per_5_days_booked", "discount_per_10_days_booked", "discount_per_30_and_more_days_booked", "service_cost"]] = \
df_list[["host_acceptance_rate", "review_scores_rating", "discount_per_5_days_booked", "discount_per_10_days_booked", "discount_per_30_and_more_days_booked", "service_cost"]].astype("float16")

In [ ]:
df_list.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4817 entries, 0 to 6172
Data columns (total 26 columns):
 #   Column                                Non-Null Count  Dtype   
---  ------                                --------------  -----   
 0   id                                    4817 non-null   int64   
 1   host_acceptance_rate                  4817 non-null   float16 
 2   host_is_superhost                     4817 non-null   bool    
 3   neighbourhood                         4817 non-null   category
 4   latitude                              4817 non-null   float64 
 5   longitude                             4817 non-null   float64 
 6   room_type                             4817 non-null   category
 7   accommodates                          4817 non-null   int8    
 8   bedrooms                              4817 non-null   int8    
 9   beds                                  4817 non-null   int8    
 10  amenities                             4817 non-null   int8    
 11  pric

We can see there that we reduced the memory usage from 852 kb to 396 kb

---

## Cleaning Is DONE!

<center>
  <img src=https://i.ibb.co/f9wQt8T/2016-celebrate-celebration-city-preview.jpg width="500" align="center" />
</center>
<br/>


In [ ]:
df_list.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4817 entries, 0 to 6172
Data columns (total 26 columns):
 #   Column                                Non-Null Count  Dtype   
---  ------                                --------------  -----   
 0   id                                    4817 non-null   int64   
 1   host_acceptance_rate                  4817 non-null   float16 
 2   host_is_superhost                     4817 non-null   bool    
 3   neighbourhood                         4817 non-null   category
 4   latitude                              4817 non-null   float64 
 5   longitude                             4817 non-null   float64 
 6   room_type                             4817 non-null   category
 7   accommodates                          4817 non-null   int8    
 8   bedrooms                              4817 non-null   int8    
 9   beds                                  4817 non-null   int8    
 10  amenities                             4817 non-null   int8    
 11  pric

Using the function [`head()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html) reveals the same. 

In [ ]:
df_list.head(3)

,id,host_acceptance_rate,host_is_superhost,neighbourhood,latitude,longitude,room_type,accommodates,bedrooms,beds,amenities,price_in_dollar,minimum_nights,maximum_nights,has_availability,availability_30,availability_60,number_of_reviews_l30d,review_scores_rating,instant_bookable,price_per_person,minimum_price,discount_per_5_days_booked,discount_per_10_days_booked,discount_per_30_and_more_days_booked,service_cost
0,23726706,0.95,False,IJburg - Zeeburgereiland,52.35,4.98,Private room,2,1,1,6,88.00,2,14,True,0,6,3,4.99,True,44.00,176.00,0.05,0.11,0.16,4.99
1,35815036,1.00,True,Noord-Oost,52.42,4.96,Entire home/apt,2,1,1,5,105.00,3,100,True,4,6,6,4.96,False,52.50,315.00,0.05,0.12,0.16,4.99
2,31553121,1.00,False,Noord-West,52.43,4.92,Entire home/apt,4,2,3,3,152.00,2,60,True,0,3,1,4.74,False,38.00,304.00,0.07,0.11,0.22,4.99


## Mix and Match

<center>
  <img src=https://successrice.com/wp-content/uploads/2020/08/Pride-038-1-980x551.jpg.webp width="500" align="center" />
</center>
<br/>

As a next step, we will merge these two datasets.

In [ ]:
# The Calendar DataFrame!
df_cal.head(3)

,listing_id,date,available,price_in_dollar,minimum_nights,maximum_nights
0,23726706,2022-06-05,False,90.00,2,1125
1,23726706,2022-06-06,False,90.00,2,1125
2,23726706,2022-06-07,False,90.00,2,1125


---

#### Minimum stay

Let us try an experiment for AIRBNB renters.

The situation is described below: 

You are looking to stay for a minimum of 3 days. Since you are unsure when it will be booked, you'd like to exclude all `listing_ids` that go below that threshold of 3 days no matter what time of year. This is because you think that bookings with a minimum stay of 3 days are more likely to have discount prices.

With these excluded, you would like to see the expected booking price for 5 days.

Steps to do that :  

- Using the helper code, we will create a conditional index for entries `listing_id` that are above the 3 day threshold
- We will remove them using `.loc[conditional_index]` over the conditional index.
- We will calculate the price of booking a listing for 5 days by multiplying the current day multiplied by 5, and assign this to a column called `five_day_dollar_price`

In [ ]:
# copy, for debugging purposes
calendar_newdf = df_cal.copy()

include_list = (
    calendar_newdf["minimum_nights"] >= 3
)

In [ ]:
# Get all the listings with a minimum nights of 3+
# Use the include_list
# calendar_newdf = df.loc[include_list]
calendar_newdf = calendar_newdf.loc[include_list]

In [ ]:
calendar_newdf.head()

,listing_id,date,available,price_in_dollar,minimum_nights,maximum_nights
365,35815036,2022-06-05,False,105.00,3,1125
367,35815036,2022-06-07,True,105.00,3,1125
368,35815036,2022-06-08,True,105.00,3,1125
369,35815036,2022-06-09,True,105.00,3,1125
370,35815036,2022-06-10,False,105.00,3,1125


| Related functions |
| ---- |
| [isin()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isin.html): Filter the DataFrame on provided values |
| [eq()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.eq.html#pandas.DataFrame.eq): Filter the DataFrame for all values equal to the provided input |
| [ne()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ne.html#pandas.DataFrame.ne): Filter the DataFrame for all values not equal to the provided input |

In [ ]:
calendar_newdf["five_day_dollar_price"] = calendar_newdf["price_in_dollar"]*5

In [ ]:
calendar_newdf.head()

,listing_id,date,available,price_in_dollar,minimum_nights,maximum_nights,five_day_dollar_price
365,35815036,2022-06-05,False,105.00,3,1125,525.00
367,35815036,2022-06-07,True,105.00,3,1125,525.00
368,35815036,2022-06-08,True,105.00,3,1125,525.00
369,35815036,2022-06-09,True,105.00,3,1125,525.00
370,35815036,2022-06-10,False,105.00,3,1125,525.00


Now let's transform our newly created DataFrame into a **pivot table**, where we aggregate our rows using the `listing_id` as the index, and the columns `available` and `five_day_dollar_price` as values.

The expected table output should look something like.

|listing\_id|available|five\_day\_dollar\_price|
|---|---|---|
|2818|0\.20821917808219179|346\.90410958904107|
|44391|0\.0|1200\.0|
|49552|0\.4581005586592179|1162\.5|


In [ ]:
calendar_newdf[calendar_newdf["listing_id"] == 2818]

,listing_id,date,available,price_in_dollar,minimum_nights,maximum_nights,five_day_dollar_price
1869165,2818,2022-06-05,False,52.00,3,1125,260.00
1869166,2818,2022-06-06,False,49.00,3,1125,245.00
1869167,2818,2022-06-07,False,49.00,3,1125,245.00
1869168,2818,2022-06-08,False,49.00,3,1125,245.00
1869169,2818,2022-06-09,False,49.00,3,1125,245.00
...,...,...,...,...,...,...,...
1869525,2818,2023-05-31,False,78.00,3,1125,390.00
1869526,2818,2023-06-01,False,78.00,3,1125,390.00
1869527,2818,2023-06-02,False,78.00,3,1125,390.00
1869528,2818,2023-06-03,False,78.00,3,1125,390.00


In [ ]:
calendar_summarizeddf = pd.pivot_table(
    data=calendar_newdf,
    index=["listing_id"],
    values=["available", "five_day_dollar_price"],
    aggfunc=np.mean,  # The default aggregation function used
    # for merging multiple related rows of data.
)

calendar_summarizeddf.head()

,available,five_day_dollar_price
listing_id,,
2818,0.21,346.90
44391,0.00,1200.00
49552,0.46,1162.50
55709,0.00,818.68
55868,0.00,749.22


#### Maximum price and date

Let us make a **pivot table** that states the **maximum `price_in_dollar`** for every Airbnb listing?

In [ ]:
temp_sum_df = pd.pivot_table(
    data = calendar_newdf,
    index = "listing_id",
    values = ["price_in_dollar"],
    aggfunc = np.max
)

temp_sum_df.head(3)

,price_in_dollar
listing_id,
2818,80.00
44391,240.00
49552,300.00


---

## Join Us?!

<center>
  <img src=https://upload.wikimedia.org/wikipedia/commons/9/95/Merge_left_%28259959%29_-_The_Noun_Project.svg width="200" align="center" />
</center>
<br/>

We are going to merge the pivot table that includes the `five_day_dollar_price` for each listing with `listings_df`. We have to keep in mind that we want to keep only those rows of Airbnb listing IDs that are present in both datasets.

#### Mergin'

In [ ]:
calendar_newdf

,listing_id,date,available,price_in_dollar,minimum_nights,maximum_nights,five_day_dollar_price
365,35815036,2022-06-05,False,105.00,3,1125,525.00
367,35815036,2022-06-07,True,105.00,3,1125,525.00
368,35815036,2022-06-08,True,105.00,3,1125,525.00
369,35815036,2022-06-09,True,105.00,3,1125,525.00
370,35815036,2022-06-10,False,105.00,3,1125,525.00
...,...,...,...,...,...,...,...
2252045,47709873,2023-05-31,False,213.00,7,120,1065.00
2252046,47709873,2023-06-01,False,213.00,7,120,1065.00
2252047,47709873,2023-06-02,False,213.00,7,120,1065.00
2252048,47709873,2023-06-03,False,213.00,7,120,1065.00


In [ ]:
calendar_summarizeddf.iloc[3733]

available                 0.08
five_day_dollar_price   570.32
Name: 640513271677293480, dtype: float64

In [ ]:
final_df = pd.merge(
    left = df_list,
    right = calendar_summarizeddf,
    how = "inner",
    left_on = "id",
    right_on = "listing_id",
)
final_df.tail()

,id,host_acceptance_rate,host_is_superhost,neighbourhood,latitude,longitude,room_type,accommodates,bedrooms,beds,amenities,price_in_dollar,minimum_nights,maximum_nights,has_availability,availability_30,availability_60,number_of_reviews_l30d,review_scores_rating,instant_bookable,price_per_person,minimum_price,discount_per_5_days_booked,discount_per_10_days_booked,discount_per_30_and_more_days_booked,service_cost,available,five_day_dollar_price
2878,557194226234491250,0.95,False,Watergraafsmeer,52.31,4.91,Entire home/apt,4,2,2,9,272.00,3,10,True,4,5,2,5.00,False,68.00,816.00,0.12,0.19,0.22,4.99,0.02,1398.29
2879,578106411434562834,0.67,False,Noord-Oost,52.43,4.95,Entire home/apt,5,3,3,13,350.00,6,365,True,0,0,0,5.00,False,70.00,2100.00,0.11,0.15,0.26,4.99,0.00,1750.00
2880,47775130,1.00,False,Buitenveldert - Zuidas,52.32,4.87,Entire home/apt,2,1,1,5,142.00,4,186,True,0,0,0,4.75,True,71.00,568.00,0.07,0.13,0.19,4.99,0.00,460.75
2881,48005583,1.00,False,Buitenveldert - Zuidas,52.32,4.87,Entire home/apt,2,1,1,3,142.00,4,180,True,0,0,0,4.50,True,71.00,568.00,0.06,0.08,0.15,4.99,0.00,566.23
2882,36900951,0.85,False,Buitenveldert - Zuidas,52.32,4.86,Entire home/apt,4,2,2,4,174.00,3,1125,True,0,2,1,4.92,False,43.50,522.00,0.07,0.10,0.22,4.99,0.01,871.58


In [ ]:
final_df.isna().any().any()

False

In [ ]:
final_df[["id", "minimum_nights", "maximum_nights"]] = final_df[["id", "minimum_nights", "maximum_nights"]].astype("int")

---

#### Groups are great

Now, let's perform a [`groupby`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) where we look at the median values of `five_day_dollar_price` and `review_scores_accuracy` with respect to the `room_type`. Do these results match your intuition?


In [ ]:
room_groups = final_df.groupby(by = ["room_type"])[["review_scores_rating", "five_day_dollar_price"]].mean()

In [ ]:
room_groups.head()

,review_scores_rating,five_day_dollar_price
room_type,,
Entire home/apt,4.79,1307.01
Hotel room,4.52,1791.05
Private room,4.70,905.94
Shared room,4.62,896.97


We might have expected that shared rooms are the cheapest and thus have the lowest rating with respect to median scores. The same can't be said for the most expensive option — a hotel room. 

(We could also think that these data might be biased in favor of Airbnb and not hotels in general. 🤷)

---

### Download of our dataset

Let's first export our final DataFrame.

In [ ]:
final_df.to_csv(
    "WK2_Airbnb_Amsterdam_listings_proj_solution.csv",
    index=True,
)

In [ ]:
from google.colab import files

# Download the file locally
files.download('WK2_Airbnb_Amsterdam_listings_proj_solution.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Visualization on Streamlit!

<center>
  <img src=https://griddb-pro.azureedge.net/en/wp-content/uploads/2021/08/streamlit-1160x650.png width="500" align="center" />
</center>
<br/>

Let us make an app that visualizes the dataset as a DataFrame and as a geographic visualization 

In [ ]:
%%writefile app.py
import pandas as pd
import streamlit as st
from pandas.api.types import (
    is_categorical_dtype,
    is_datetime64_any_dtype,
    is_numeric_dtype,
    is_object_dtype
)

st.title("Filter your Airbnb Listings dataframe!")

st.write(
    """This app is based on this blog [here](https://blog.streamlit.io/auto-generate-a-dataframe-filtering-ui-in-streamlit-with-filter_dataframe/). 
    Can you think of ways to extend it with visuals?
    """
)


def filter_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """
    Adds a UI on top of a dataframe to let viewers filter columns
    Args:
        df (pd.DataFrame): Original dataframe
    Returns:
        pd.DataFrame: Filtered dataframe
    """
    modify = st.checkbox("Add filters")

    if not modify:
        return df

    df = df.copy()

    # Try to convert datetimes into a standard format (datetime, no timezone)
    for col in df.columns:
        if is_object_dtype(df[col]):
            try:
                df[col] = pd.to_datetime(df[col])
            except Exception:
                pass

        if is_datetime64_any_dtype(df[col]):
            df[col] = df[col].dt.tz_localize(None)

    modification_container = st.container()

    with modification_container:
        to_filter_columns = st.multiselect("Filter dataframe on", df.columns)
        for column in to_filter_columns:
            left, right = st.columns((1, 20))
            left.write("↳")
            # Treat columns with < 10 unique values as categorical
            if is_categorical_dtype(df[column]) or df[column].nunique() < 10:
                user_cat_input = right.multiselect(
                    f"Values for {column}",
                    df[column].unique(),
                    default=list(df[column].unique()),
                )
                df = df[df[column].isin(user_cat_input)]
            elif is_numeric_dtype(df[column]):
                _min = float(df[column].min())
                _max = float(df[column].max())
                step = (_max - _min) / 100
                user_num_input = right.slider(
                    f"Values for {column}",
                    _min,
                    _max,
                    (_min, _max),
                    step=step,
                )
                df = df[df[column].between(*user_num_input)]
            elif is_datetime64_any_dtype(df[column]):
                user_date_input = right.date_input(
                    f"Values for {column}",
                    value=(
                        df[column].min(),
                        df[column].max(),
                    ),
                )
                if len(user_date_input) == 2:
                    user_date_input = tuple(map(pd.to_datetime, user_date_input))
                    start_date, end_date = user_date_input
                    df = df.loc[df[column].between(start_date, end_date)]
            else:
                user_text_input = right.text_input(
                    f"Substring or regex in {column}",
                )
                if user_text_input:
                    df = df[df[column].str.contains(user_text_input)]

    return df


df = pd.read_csv(
    "WK2_Airbnb_Amsterdam_listings_proj_solution.csv", index_col=0
)
st.dataframe(filter_dataframe(df))

Writing app.py


The **%%writefile [FILE_NAME].[FILE_EXTENSION]** command let's us save the code written in the cells in the Google Colab instance. Having it saved like that enables us to download it as a file, as seen below.

In [ ]:
from google.colab import files

# Download the file locally
files.download('app.py')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
%%writefile requirements.txt
pandas
streamlit

Writing requirements.txt


In [ ]:
from google.colab import files

# Download the file locally
files.download('requirements.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>